In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import io
from termcolor import colored

In [2]:
df=pd.read_csv('BHARTITEST.csv',index_col=[0])

In [3]:
df['DateTime']=pd.to_datetime(df['Date']+" "+df['Time'])
df=df.reindex(columns=[ 'DateTime','Date', 'Time', 'Open', 'High', 'Low', 'Close'])
df.drop(['Date','Time'],axis=1,inplace=True)

In [4]:
df.head()

,DateTime,Open,High,Low,Close
564885,2020-03-01 09:15:59,457.25,457.95,455.00,455.30
564886,2020-03-01 09:16:59,455.05,456.20,455.05,456.20
564887,2020-03-01 09:17:59,456.20,456.20,454.55,454.75
564888,2020-03-01 09:18:59,454.55,455.10,454.55,455.10
564889,2020-03-01 09:19:59,455.30,456.25,455.30,455.65


In [5]:
df2=pd.read_csv('BHARTITRAIN.csv',index_col=[0])

In [6]:
df2.reset_index(drop=True,inplace=True)

In [7]:
df2

,DateTime,Open,High,Low,Close
0,2020-01-01 12:45:59,452.05,462.55,451.55,452.40
1,2020-01-01 12:50:59,452.40,462.55,451.55,452.80
2,2020-01-01 12:55:59,452.80,462.55,451.55,452.50
3,2020-01-01 13:00:59,452.50,462.55,451.55,452.75
4,2020-01-01 13:05:59,453.00,462.55,451.55,453.00
...,...,...,...,...,...
95,2020-02-01 15:05:59,457.80,462.55,451.55,458.60
96,2020-02-01 15:10:59,458.60,462.55,451.55,457.70
97,2020-02-01 15:15:59,457.70,462.55,451.55,458.10
98,2020-02-01 15:20:59,458.10,462.55,451.55,458.60


In [8]:
lst=df2['Close'].to_list()

In [9]:
last10=lst[-10:]
last10

[458.1, 458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5]

In [10]:
long_target=1
long_sl=-0.5

short_target=-1
short_sl=0.5
df['SMA']=0.0

In [11]:
dates=['03/01/2020', '06/01/2020',
       '07/01/2020', '08/01/2020', '13/01/2020', '14/01/2020',
       '15/01/2020', '16/01/2020', '17/01/2020', '20/01/2020',
       '21/01/2020', '22/01/2020', '24/01/2020', '27/01/2020',
       '29/01/2020', '30/01/2020']
prevcandles=pd.read_csv('5minCandle1-30.csv',index_col=[0])

In [12]:
prevcandles.head(176)

,DateTime,Open,High,Low,Close
0,2020-01-01 12:45:59,452.05,462.55,451.55,452.40
1,2020-01-01 12:50:59,452.40,462.55,451.55,452.80
2,2020-01-01 12:55:59,452.80,462.55,451.55,452.50
3,2020-01-01 13:00:59,452.50,462.55,451.55,452.75
4,2020-01-01 13:05:59,453.00,462.55,451.55,453.00
...,...,...,...,...,...
171,2020-03-01 15:25:59,456.10,458.60,456.05,458.45
172,2020-06-01 09:15:59,455.45,458.65,455.20,458.10
173,2020-06-01 09:20:59,457.95,459.00,454.75,455.35
174,2020-06-01 09:25:59,455.65,456.50,454.10,454.10


In [13]:
date=[]
for i in dates:
    date.append(str(pd.to_datetime(i[:10]))[:10])
date

['2020-03-01',
 '2020-06-01',
 '2020-07-01',
 '2020-08-01',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-24',
 '2020-01-27',
 '2020-01-29',
 '2020-01-30']

In [14]:
s='2020-03-01 15:09:59'
k=pd.to_datetime(s[:10])
str(k)[:10]

'2020-03-01'

In [15]:
tradesheet=pd.DataFrame(columns=['EntryTime','ExitTime','TradeType','EntryPrice','ExitPrice','Spread'])

In [17]:
def get_5min_dfs(starttime):
    res=[]
    for i in range(len(prevcandles)):
        if prevcandles['DateTime'].iloc[i]==starttime:
            k=i-9
            for itr in range(k,i+1):
                res.append(prevcandles['Close'].iloc[itr])
    return res
def func(dt):
    end=""
    start=""
    dt=str(dt)
    ind=date.index(dt[:10])
    start+=date[ind+1]+" "
    end+=date[ind+1]+" "
    start+='09:30:59'
    end+='15:09:59'
    return [start,end]

import sys
from datetime import timedelta
from datetime import datetime
hashmap={0:[last10[-1],sum(last10)/len(last10)]}
longt=0
longsl=0
shortt=0
shortsl=0
high=0
low=10000007
trade=0
flag=0
close=455.30
ltarget=-1
starget=-1
sstoploss=-1
lstoploss=-1
start=0

starttime=datetime.fromisoformat('2020-03-01 09:30:59')
endtime=datetime.fromisoformat('2020-03-01 15:09:59')


for i,j in enumerate(zip(df.Close,df.SMA,df.High,df.Low,df.DateTime,df.Open)):
    tradcount=0
    res=[*j]
    sma=0
    print("___ DATA ___")
    print('DATETIME:',res[4])
    print("OPEN:",res[5])
    print("HIGH:",res[2])
    print("LOW",res[3])
    print("CLOSE",res[0])
    print("------------------")
    
    #long entry short exit-> green 
    #long exit short entry -> red
  
    #df.Close 0 ,df.SMA10 1,df.High 2,df.Low 3 ,df.DateTime,df.Open
    
    if df.DateTime.iloc[i]==starttime:
        del last10
        last10=get_5min_dfs(str(starttime))
        print("gen",last10)
        start=1
        print(colored("TRADING STARTED",'green'))
        
    if df.DateTime.iloc[i]==endtime:
        print(colored("SQUARED OFF !",'red'))
        print("Close :",res[0])
        try:
            starttime=datetime.fromisoformat(func(endtime)[0])
            endtime=datetime.fromisoformat(func(endtime)[1])
        except:
            print("End of trading")
        print(starttime,endtime)
        start=0
        flag=0
        trade=0
        
    if res[4].minute%5==0:
        dx={}
        last10.pop(0)
        last10.append(res[0])
        sma=sum(last10)/len(last10)
        print(last10,res[0])
        print("SMA-->",sma)
        df.iat[i, df.columns.get_loc('SMA')]=sma
        dx={'DateTime':res[4],
             'Open':res[5]}
        high=res[2]
        low=res[3]
        close=res[0]
    
    elif res[4].minute%5!=0:
        last10[-1]=res[0]
        sma=sum(last10)/len(last10)
        #print("n",res[3],last10)
        print(last10,res[0])
        print("SMA-->",sma)
        df.iat[i, df.columns.get_loc('SMA')]=sma 
        
        high=max(high,res[2])
        low=min(low,res[3])
        close=res[0]
        if (df['DateTime'].iloc[i]+timedelta(minutes=1)).minute%5==0:
            #print(df['DateTime'].iloc[i])
            dx['High']=high
            dx['Low']=low
            dx['Close']=close
            print()
            print("Pushing Data")
            print(dx)
            #df2=df2.append(dx,ignore_index=True)

    
        
      
    if trade==0 and start==1 and res[0]>sma and hashmap[i][0]<hashmap[i][1]:
        print(colored("LONG ENTRY CONDITION SATISFIED !",'green'))
        trade=1
        tradcount+=1
        flag=1
        print("1. Close :{} > SMA: {}".format(res[0],sma))
        print("2. PrevClose :{} < PrevSMA :{}".format(hashmap[i][0],hashmap[i][1]))
        print()
        print("BUY BHARTIAIRTEL AT ",res[4],"IN Rs",res[0])
        ltarget=res[0]+res[0]*long_target/100
        lstoploss=res[0]+res[0]*long_sl/100
        print('Target->',ltarget,'Stoploss->',lstoploss)
        
        data={'EntryTime':df['DateTime'].iloc[i],'ExitTime':None,'TradeType':'LONG','EntryPrice':df['Close'].iloc[i],'ExitPrice':None,'Spread':None}
        print()
    if tradcount==1:
        tradcount+=1
        continue
        
        
    if res[2]>ltarget and flag==1:
        print(colored('Target Achieved- Long Exit','red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[2]
        data['Spread']=data['ExitPrice']-data['EntryPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
        tradcount=0
    elif res[3]<lstoploss and flag==1:
        print(colored("Stoploss Achieved- Long Exit",'red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[3]
        data['Spread']=data['ExitPrice']-data['EntryPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
        tradcount=0


    if trade==0 and start==1 and res[0]<sma and hashmap[i][0]>hashmap[i][1]:
        trade=1
        tradcount+=1
        flag=2
        print(colored("SHORT ENTRY CONDITION SATISFIED !",'red'))
        print("1. Close :{} < SMA: {}".format(res[0],sma))
        print("2. PrevClose :{} > PrevSMA :{}".format(hashmap[i][0],hashmap[i][1]))
        print()
        print("SELL BHARTIAIRTEL AT ",res[4],"IN Rs",res[0])
        starget=res[0]+res[0]*short_target/100
        sstoploss=res[0]+res[0]*short_sl/100
        print('Target->',starget,'Stoploss->',sstoploss)
        data={'EntryTime':df['DateTime'].iloc[i],'ExitTime':0,'TradeType':'SHORT','EntryPrice':df['Close'].iloc[i],'ExitPrice':0,'Spread':0}
        print()
    if tradcount==1:
        tradcount+=1
        continue

    if res[3]<starget and flag==2:
        print(colored('Short_Target Achieved','green'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[3]
        data['Spread']=data['EntryPrice']-data['ExitPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
    elif res[2]>sstoploss and flag==2:
        print(colored("Short_Stoploss Achieved",'red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[2]
        data['Spread']=data['EntryPrice']-data['ExitPrice']
        
        tradesheet=tradesheet.append(data,ignore_index=True)
        
        data={}
        trade=0
        flag=0

        
        
    print(trade,flag,tradcount)
    hashmap[i+1]=[res[0],sma]
            
        

___ DATA ___
DATETIME: 2020-03-01 09:15:59
OPEN: 457.25
HIGH: 457.95
LOW 455.0
CLOSE 455.3
------------------
[458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 455.3] 455.3
SMA--> 457.98999999999995
0 0 0
___ DATA ___
DATETIME: 2020-03-01 09:16:59
OPEN: 455.05
HIGH: 456.2
LOW 455.05
CLOSE 456.2
------------------
[458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 456.2] 456.2
SMA--> 458.0799999999999
0 0 0
___ DATA ___
DATETIME: 2020-03-01 09:17:59
OPEN: 456.2
HIGH: 456.2
LOW 454.55
CLOSE 454.75
------------------
[458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 454.75] 454.75
SMA--> 457.93499999999995
0 0 0
___ DATA ___
DATETIME: 2020-03-01 09:18:59
OPEN: 454.55
HIGH: 455.1
LOW 454.55
CLOSE 455.1
------------------
[458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 455.1] 455.1
SMA--> 457.96999999999997
0 0 0
___ DATA ___
DATETIME: 2020-03-01 09:19:59
OPEN: 455.3
HIGH: 456.25
LOW 455.3
CLOSE 455.65
------------------
[458.

0 0 0
___ DATA ___
DATETIME: 2020-06-01 11:02:59
OPEN: 450.8
HIGH: 451.0
LOW 450.75
CLOSE 451.0
------------------
[452.0, 451.9, 451.95, 451.8, 452.15, 451.8, 452.05, 451.85, 450.65, 451.0] 451.0
SMA--> 451.715
0 0 0
___ DATA ___
DATETIME: 2020-06-01 11:03:59
OPEN: 450.65
HIGH: 450.65
LOW 450.6
CLOSE 450.6
------------------
[452.0, 451.9, 451.95, 451.8, 452.15, 451.8, 452.05, 451.85, 450.65, 450.6] 450.6
SMA--> 451.675
0 0 0
___ DATA ___
DATETIME: 2020-06-01 11:04:59
OPEN: 450.8
HIGH: 451.0
LOW 450.8
CLOSE 451.0
------------------
[452.0, 451.9, 451.95, 451.8, 452.15, 451.8, 452.05, 451.85, 450.65, 451.0] 451.0
SMA--> 451.715

Pushing Data
{'DateTime': Timestamp('2020-06-01 11:00:59'), 'Open': 450.65, 'High': 451.0, 'Low': 449.5, 'Close': 451.0}
0 0 0
___ DATA ___
DATETIME: 2020-06-01 11:05:59
OPEN: 451.15
HIGH: 451.5
LOW 451.1
CLOSE 451.35
------------------
[451.9, 451.95, 451.8, 452.15, 451.8, 452.05, 451.85, 450.65, 451.0, 451.35] 451.35
SMA--> 451.65
0 0 0
___ DATA ___
DATETIME:

0 0 0
___ DATA ___
DATETIME: 2020-08-01 13:38:59
OPEN: 454.1
HIGH: 454.1
LOW 453.7
CLOSE 453.7
------------------
[452.0, 452.9, 452.25, 453.25, 452.35, 451.7, 452.65, 452.55, 453.3, 453.7] 453.7
SMA--> 452.6650000000001
0 0 0
___ DATA ___
DATETIME: 2020-08-01 13:39:59
OPEN: 453.7
HIGH: 454.0
LOW 453.7
CLOSE 453.8
------------------
[452.0, 452.9, 452.25, 453.25, 452.35, 451.7, 452.65, 452.55, 453.3, 453.8] 453.8
SMA--> 452.675

Pushing Data
{'DateTime': Timestamp('2020-08-01 13:35:59'), 'Open': 453.3, 'High': 454.55, 'Low': 453.05, 'Close': 453.8}
0 0 0
___ DATA ___
DATETIME: 2020-08-01 13:40:59
OPEN: 453.8
HIGH: 454.15
LOW 453.8
CLOSE 454.15
------------------
[452.9, 452.25, 453.25, 452.35, 451.7, 452.65, 452.55, 453.3, 453.8, 454.15] 454.15
SMA--> 452.89000000000004
0 0 0
___ DATA ___
DATETIME: 2020-08-01 13:41:59
OPEN: 454.15
HIGH: 454.15
LOW 453.8
CLOSE 454.0
------------------
[452.9, 452.25, 453.25, 452.35, 451.7, 452.65, 452.55, 453.3, 453.8, 454.0] 454.0
SMA--> 452.875
0 0 0


___ DATA ___
DATETIME: 2020-01-15 09:37:59
OPEN: 469.6
HIGH: 469.7
LOW 469.45
CLOSE 469.5
------------------
[469.1, 468.9, 468.95, 469.0, 468.65, 471.65, 471.05, 469.35, 469.35, 469.5] 469.5
SMA--> 469.55
1 2 0
___ DATA ___
DATETIME: 2020-01-15 09:38:59
OPEN: 469.5
HIGH: 469.75
LOW 469.0
CLOSE 469.1
------------------
[469.1, 468.9, 468.95, 469.0, 468.65, 471.65, 471.05, 469.35, 469.35, 469.1] 469.1
SMA--> 469.51000000000005
1 2 0
___ DATA ___
DATETIME: 2020-01-15 09:39:59
OPEN: 469.1
HIGH: 469.1
LOW 468.55
CLOSE 468.9
------------------
[469.1, 468.9, 468.95, 469.0, 468.65, 471.65, 471.05, 469.35, 469.35, 468.9] 468.9
SMA--> 469.48999999999995

Pushing Data
{'DateTime': Timestamp('2020-01-15 09:35:59'), 'Open': 469.35, 'High': 469.75, 'Low': 468.55, 'Close': 468.9}
1 2 0
___ DATA ___
DATETIME: 2020-01-15 09:40:59
OPEN: 469.15
HIGH: 469.35
LOW 468.9
CLOSE 469.1
------------------
[468.9, 468.95, 469.0, 468.65, 471.65, 471.05, 469.35, 469.35, 468.9, 469.1] 469.1
SMA--> 469.490000000000

[498.4, 499.0, 499.1, 498.7, 501.05, 500.55, 499.9, 499.55, 499.2, 499.4] 499.4
SMA--> 499.485
1 2 0
___ DATA ___
DATETIME: 2020-01-17 14:22:59
OPEN: 499.4
HIGH: 499.55
LOW 499.2
CLOSE 499.2
------------------
[498.4, 499.0, 499.1, 498.7, 501.05, 500.55, 499.9, 499.55, 499.2, 499.2] 499.2
SMA--> 499.46500000000003
1 2 0
___ DATA ___
DATETIME: 2020-01-17 14:23:59
OPEN: 499.2
HIGH: 499.4
LOW 499.2
CLOSE 499.3
------------------
[498.4, 499.0, 499.1, 498.7, 501.05, 500.55, 499.9, 499.55, 499.2, 499.3] 499.3
SMA--> 499.4750000000001
1 2 0
___ DATA ___
DATETIME: 2020-01-17 14:24:59
OPEN: 499.3
HIGH: 499.3
LOW 498.8
CLOSE 499.15
------------------
[498.4, 499.0, 499.1, 498.7, 501.05, 500.55, 499.9, 499.55, 499.2, 499.15] 499.15
SMA--> 499.46000000000004

Pushing Data
{'DateTime': Timestamp('2020-01-17 14:20:59'), 'Open': 499.2, 'High': 499.55, 'Low': 498.8, 'Close': 499.15}
1 2 0
___ DATA ___
DATETIME: 2020-01-17 14:25:59
OPEN: 499.25
HIGH: 499.25
LOW 498.85
CLOSE 498.95
------------------
[

TRADING STARTED
[513.35, 510.55, 511.1, 511.6, 509.8, 517.0, 515.65, 515.4, 514.55, 514.05] 514.05
SMA--> 513.3050000000001
0 0 0
___ DATA ___
DATETIME: 2020-01-22 09:31:59
OPEN: 514.05
HIGH: 514.25
LOW 513.6
CLOSE 513.6
------------------
[513.35, 510.55, 511.1, 511.6, 509.8, 517.0, 515.65, 515.4, 514.55, 513.6] 513.6
SMA--> 513.26
0 0 0
___ DATA ___
DATETIME: 2020-01-22 09:32:59
OPEN: 513.3
HIGH: 515.1
LOW 513.2
CLOSE 514.8
------------------
[513.35, 510.55, 511.1, 511.6, 509.8, 517.0, 515.65, 515.4, 514.55, 514.8] 514.8
SMA--> 513.38
0 0 0
___ DATA ___
DATETIME: 2020-01-22 09:33:59
OPEN: 514.8
HIGH: 515.2
LOW 514.65
CLOSE 515.0
------------------
[513.35, 510.55, 511.1, 511.6, 509.8, 517.0, 515.65, 515.4, 514.55, 515.0] 515.0
SMA--> 513.4
0 0 0
___ DATA ___
DATETIME: 2020-01-22 09:34:59
OPEN: 514.95
HIGH: 515.0
LOW 514.4
CLOSE 514.55
------------------
[513.35, 510.55, 511.1, 511.6, 509.8, 517.0, 515.65, 515.4, 514.55, 514.55] 514.55
SMA--> 513.355

Pushing Data
{'DateTime': Timest

0 0 0
___ DATA ___
DATETIME: 2020-01-27 13:36:59
OPEN: 518.35
HIGH: 518.35
LOW 517.2
CLOSE 517.25
------------------
[520.05, 520.55, 520.7, 519.9, 519.6, 518.95, 519.3, 519.05, 518.6, 517.25] 517.25
SMA--> 519.3950000000001
0 0 0
___ DATA ___
DATETIME: 2020-01-27 13:37:59
OPEN: 517.25
HIGH: 517.45
LOW 517.1
CLOSE 517.3
------------------
[520.05, 520.55, 520.7, 519.9, 519.6, 518.95, 519.3, 519.05, 518.6, 517.3] 517.3
SMA--> 519.4000000000001
0 0 0
___ DATA ___
DATETIME: 2020-01-27 13:38:59
OPEN: 517.3
HIGH: 517.35
LOW 517.1
CLOSE 517.35
------------------
[520.05, 520.55, 520.7, 519.9, 519.6, 518.95, 519.3, 519.05, 518.6, 517.35] 517.35
SMA--> 519.4050000000001
0 0 0
___ DATA ___
DATETIME: 2020-01-27 13:39:59
OPEN: 517.35
HIGH: 517.55
LOW 517.1
CLOSE 517.55
------------------
[520.05, 520.55, 520.7, 519.9, 519.6, 518.95, 519.3, 519.05, 518.6, 517.55] 517.55
SMA--> 519.4250000000001

Pushing Data
{'DateTime': Timestamp('2020-01-27 13:35:59'), 'Open': 518.6, 'High': 518.6, 'Low': 517.1,

0 0 0
___ DATA ___
DATETIME: 2020-01-30 14:32:59
OPEN: 489.5
HIGH: 489.5
LOW 489.2
CLOSE 489.2
------------------
[486.55, 486.7, 487.5, 487.45, 489.0, 487.45, 487.65, 486.95, 488.75, 489.2] 489.2
SMA--> 487.71999999999997
0 0 0
___ DATA ___
DATETIME: 2020-01-30 14:33:59
OPEN: 489.2
HIGH: 489.7
LOW 489.2
CLOSE 489.7
------------------
[486.55, 486.7, 487.5, 487.45, 489.0, 487.45, 487.65, 486.95, 488.75, 489.7] 489.7
SMA--> 487.77
0 0 0
___ DATA ___
DATETIME: 2020-01-30 14:34:59
OPEN: 489.7
HIGH: 489.7
LOW 489.55
CLOSE 489.55
------------------
[486.55, 486.7, 487.5, 487.45, 489.0, 487.45, 487.65, 486.95, 488.75, 489.55] 489.55
SMA--> 487.755

Pushing Data
{'DateTime': Timestamp('2020-01-30 14:30:59'), 'Open': 488.75, 'High': 489.7, 'Low': 488.75, 'Close': 489.55}
0 0 0
___ DATA ___
DATETIME: 2020-01-30 14:35:59
OPEN: 489.5
HIGH: 489.5
LOW 488.8
CLOSE 489.0
------------------
[486.7, 487.5, 487.45, 489.0, 487.45, 487.65, 486.95, 488.75, 489.55, 489.0] 489.0
SMA--> 488.0
0 0 0
___ DATA _

In [19]:
tradesheet

,EntryTime,ExitTime,TradeType,EntryPrice,ExitPrice,Spread
0,2020-03-01 09:43:59,2020-03-01 11:51:59,LONG,457.15,454.85,-2.3
1,2020-03-01 12:40:59,2020-03-01 14:25:59,LONG,454.8,452.3,-2.5
2,2020-06-01 10:30:59,2020-06-01 11:01:59,LONG,452.2,449.5,-2.7
3,2020-06-01 11:07:59,2020-06-01 11:48:59,LONG,451.8,449.05,-2.75
4,2020-07-01 09:53:59,2020-07-01 11:12:59,SHORT,454.3,449.6,4.7
5,2020-07-01 12:13:59,2020-07-01 13:08:59,LONG,448.8,446.45,-2.35
6,2020-08-01 09:30:59,2020-08-01 10:04:59,LONG,447.0,444.55,-2.45
7,2020-08-01 10:31:59,2020-08-01 10:49:59,LONG,445.2,442.65,-2.55
8,2020-08-01 10:55:59,2020-08-01 12:01:59,LONG,445.3,449.8,4.5
9,2020-08-01 13:15:59,2020-08-01 13:37:59,SHORT,452.1,454.55,-2.45


In [20]:
df1=tradesheet.copy()

In [21]:
qty = 100
initial_cap=100000
pnl = df1["Spread"].tolist()
df1["pnl"]=df1["Spread"] * qty
df1["Balance"] = 0
df1["cumm_ret"] = 0
df1["strategy_return"] = 0

for i in range(len(df1)):
    if(i == 0): 
        df1["Balance"].iloc[i] = df1["pnl"].iloc[i] + initial_cap
    else:
        df1["Balance"].iloc[i] = df1["pnl"].iloc[i] + df1["Balance"].iloc[i-1]
        
for i in range(len(df1)):
    if(i == 0):
        df1["strategy_return"].iloc[i] = (df1["Balance"].iloc[i] - initial_cap)/initial_cap
    else:
        df1["strategy_return"].iloc[i] = (df1["Balance"].iloc[i] - df1["Balance"].iloc[i-1])/df1["Balance"].iloc[i-1]

for i in range(len(df1)):
    df1["cumm_ret"].iloc[i] = (df1["Balance"].iloc[i] - initial_cap)/initial_cap

In [22]:
df1

,EntryTime,ExitTime,TradeType,EntryPrice,ExitPrice,Spread,pnl,Balance,cumm_ret,strategy_return
0,2020-03-01 09:43:59,2020-03-01 11:51:59,LONG,457.15,454.85,-2.3,-230.0,99770.0,-0.00230,-0.002300
1,2020-03-01 12:40:59,2020-03-01 14:25:59,LONG,454.8,452.3,-2.5,-250.0,99520.0,-0.00480,-0.002506
2,2020-06-01 10:30:59,2020-06-01 11:01:59,LONG,452.2,449.5,-2.7,-270.0,99250.0,-0.00750,-0.002713
3,2020-06-01 11:07:59,2020-06-01 11:48:59,LONG,451.8,449.05,-2.75,-275.0,98975.0,-0.01025,-0.002771
4,2020-07-01 09:53:59,2020-07-01 11:12:59,SHORT,454.3,449.6,4.7,470.0,99445.0,-0.00555,0.004749
5,2020-07-01 12:13:59,2020-07-01 13:08:59,LONG,448.8,446.45,-2.35,-235.0,99210.0,-0.00790,-0.002363
6,2020-08-01 09:30:59,2020-08-01 10:04:59,LONG,447.0,444.55,-2.45,-245.0,98965.0,-0.01035,-0.002470
7,2020-08-01 10:31:59,2020-08-01 10:49:59,LONG,445.2,442.65,-2.55,-255.0,98710.0,-0.01290,-0.002577
8,2020-08-01 10:55:59,2020-08-01 12:01:59,LONG,445.3,449.8,4.5,450.0,99160.0,-0.00840,0.004559
9,2020-08-01 13:15:59,2020-08-01 13:37:59,SHORT,452.1,454.55,-2.45,-245.0,98915.0,-0.01085,-0.002471


In [23]:
pos = 0
neg = 0
for i in range(len(pnl)):
    if(pnl[i]>0):
        pos = pos + 1
    else:
        neg = neg + 1
pnl = sum(pnl) * qty
final_cap=initial_cap+pnl
final_cap

95930.0

In [24]:
import numpy as np

In [25]:
risk_free_rate = 0.04/20

sharpe = np.sqrt(20)*(np.mean(df1.strategy_return) -(risk_free_rate))/np.std(df1.strategy_return)

In [26]:
sharpe

-4.108917884005245

In [27]:
new_cum_strategy_returns = (df1['strategy_return']+1).cumprod()
# Total number of trading days
days = len(new_cum_strategy_returns)

# Calculate compounded annual growth rate
cagr = (new_cum_strategy_returns.iloc[-1]**(20/days)-1)*100
print('The CAGR of strategy is %.2f' % cagr + '%')

The CAGR of strategy is -2.22%


In [28]:
def calc_drawdown(cum_rets):
    # Calculate the running maximum
    running_max = np.maximum.accumulate(cum_rets.dropna())

    # Ensure the value never drops below 1
    running_max[running_max < 1] = 1

    # Calculate the percentage drawdown
    drawdown = (cum_rets)/running_max - 1

    # Calculate maximum drawdown
    max_dd = drawdown.min()*100

    return drawdown, max_dd

drawdown, max_dd = calc_drawdown(new_cum_strategy_returns)

In [29]:
from tabulate import tabulate

stats = tabulate(
        [
            ['Initial Capital', initial_cap],
            ['Ending Capital', round(final_cap, 2)],
            ['Num of positive trades', pos],
            ['Num of negative trades', neg],
            ['Net profit', round(pnl, 2)],
            ['Sharpe Ratio', str(round(sharpe, 2)) + " %"],
            ['CAGR', str(round(cagr, 2)) + " %"],
            ['MDD', str(round(max_dd, 2)) + " %"],
            ['CAGR/MDD', str(round(abs(cagr / max_dd), 2)) + " %"],
        ],
        headers=['Stats Name','Short Straddle'], tablefmt='orgtbl')

print(stats)

| Stats Name             | Short Straddle   |
|------------------------+------------------|
| Initial Capital        | 100000           |
| Ending Capital         | 95930.0          |
| Num of positive trades | 8                |
| Num of negative trades | 29               |
| Net profit             | -4070.0          |
| Sharpe Ratio           | -4.11 %          |
| CAGR                   | -2.22 %          |
| MDD                    | -4.14 %          |
| CAGR/MDD               | 0.54 %           |
